# Quality Check

In this notebook, we check whether there are files for which the mp3 download has failed. This is done by comparing the runtime on our filesystem to the runtime on the website. A small discrepency of up to 2 seconds is allowd.

In [1]:
import os
import pandas as pd
import ffmpeg
import multiprocessing as mp
import urllib.request
import time

In [ ]:
# Set magic number 
dt = 1/60*2 # up to 2 seconds of runtime difference is accepted.

# Load data file

In [36]:
data = pd.read_pickle("../extract_data/data.pickle")

In [37]:
data

,titles,sources,date,duration,episode,mp3_path,txt_path
0,#311 - Maartens wens voor hij doodgaat,https://anchor.fm/s/21c734c4/podcast/play/6088...,2022-11-19,00:37:04,311,../data/audio/#311 - Maartens wens voor hij do...,../data/text (copy)/file:#311 - Maartens wens ...
1,#310 - Aanval op Kremlin,https://anchor.fm/s/21c734c4/podcast/play/6076...,2022-11-17,00:31:36,310,../data/audio/#310 - Aanval op Kremlin.mp3,../data/text (copy)/file:#310 - Aanval op Krem...
2,Maarten en Tom op RTL 4,https://anchor.fm/s/21c734c4/podcast/play/6069...,2022-11-16,00:09:31,-9999,../data/audio/Maarten en Tom op RTL 4.mp3,../data/text (copy)/file:Maarten en Tom op RTL...
3,#309 - VVD-implosie,https://anchor.fm/s/21c734c4/podcast/play/6064...,2022-11-15,00:36:56,309,../data/audio/#309 - VVD-implosie.mp3,../data/text (copy)/file:#309 - VVD-implosie.m...
4,#308 - Trump gaat eraan!,https://anchor.fm/s/21c734c4/podcast/play/6055...,2022-11-13,00:39:50,308,../data/audio/#308 - Trump gaat eraan!.mp3,../data/text (copy)/file:#308 - Trump gaat era...
...,...,...,...,...,...,...,...
320,#4 - Maarten van Rossem drinkt elke nacht bier,https://anchor.fm/s/21c734c4/podcast/play/1945...,2020-09-11,00:51:17,4,../data/audio/#4 - Maarten van Rossem drinkt e...,../data/text (copy)/file:#4 - Maarten van Ross...
321,#3 - Je moet strakke dames van middelbare leef...,https://anchor.fm/s/21c734c4/podcast/play/1485...,2020-06-07,00:32:10,3,../data/audio/#3 - Je moet strakke dames van m...,../data/text (copy)/file:#3 - Je moet strakke ...
322,#2 - Heerlijk! Een zomer zonder festivals,https://anchor.fm/s/21c734c4/podcast/play/1485...,2020-06-07,00:29:11,2,../data/audio/#2 - Heerlijk! Een zomer zonder ...,../data/text (copy)/file:#2 - Heerlijk! Een zo...
323,#1 - De terminale arrogantie van Wopke Hoekstra,https://anchor.fm/s/21c734c4/podcast/play/1485...,2020-06-07,00:25:14,1,../data/audio/#1 - De terminale arrogantie van...,../data/text (copy)/file:#1 - De terminale arr...


# Extract file length from .mp3

In [38]:
data["duration-m-file"] = data["mp3_path"].transform(lambda f: float(ffmpeg.probe(f)['format']['duration'])/60)

In [39]:
data["duration-s"] = pd.to_timedelta(data["duration"]).dt.total_seconds()
data["duration-m"] = data["duration-s"]/60.0

# Get difference in podcast length and downloaded length

To filter out the files which were not downloaded correctly, compare the length as displayed on the website with the length we found for the downloaded files

In [40]:
data["length-diff-m"] = abs(data["duration-m-file"] - data["duration-m"])

In [41]:
data_incomplete = data[data["length-diff-m"] > dt]
data_incomplete

,titles,sources,date,duration,episode,mp3_path,txt_path,duration-m-file,duration-s,duration-m,length-diff-m


# Download incomplete files again

In [29]:
use_cores = mp.cpu_count()

In [30]:
def download_mp3(source, title):
    path = f"../data/audio/{title}.mp3"
    urllib.request.urlretrieve(source, path)

In [31]:
pool = mp.Pool(use_cores)
result = pool.starmap(download_mp3, tuple(zip(data_incomplete["sources"], data_incomplete["titles"])))

NameError: name 'time' is not defined

In [32]:
data_incomplete.to_pickle("incomplete_mp3_files.pickle")

In [35]:
with open("incomplete_mp3.txt","w") as f:
    for mp3 in data_incomplete["mp3_path"]:
        path = mp3.split("audio/")[1]
        f.write(f"{path}\n")